# Análise dos dados

## Sobre a análise

Temos três tipos de dados neste projeto:

1. Dados de treino
2. Dados de validação
3. Itens complementares

Os dados de treino e de validação representam informações sobre os usuários do G1, falando sobre o tipo de usuário e o histórico de acesso de notícias dele.\
Já os itens complementares são dados sobre as páginas de notícias do G1: data de publicação e o conteúdo da notícia (título, subtítulo e corpo).

O objetivo neste notebook é responder algumas perguntas sobre os dados, a fim de descobrir quais tratamentos precisaremos realizar e quais features novas poderemos criar.

Ao final, iremos mostrar as conclusões que chegamos através da análise de cada um dos tipos de dados.

## Configuração

In [62]:
from pathlib import Path

import pandas as pd

Temos vários tipos de dados que são considerados strings e queremos utilizar o tipo correto para processá-los, por isso vamos utilizar o `infer_string` do Pandas. Isso irá fazer com que as strings recebam o tipo `"string[pyarrow]"`, que processa as strings de uma forma mais eficiente do que o NumPy.

In [63]:
pd.options.future.infer_string = True  # type: ignore[attr-defined]

In [64]:
import pyarrow.csv as pv


def read_csv_pyarrow(path: Path) -> pd.DataFrame:
    """Read a CSV file with PyArrow as backend.

    Args:
        path (Path): the CSV file path

    Returns:
        pd.DataFrame: a Pandas DataFrame

    """
    try:
        return pd.read_csv(
            path,
            engine="pyarrow",
            dtype_backend="pyarrow",
        )
    except pd.errors.ParserError:
        return pv.read_csv(
            path,
            parse_options=pv.ParseOptions(newlines_in_values=True),
        ).to_pandas(types_mapper=pd.ArrowDtype)

## Análise dos dados de treino

In [4]:
train_path = "../data/raw/files/treino/*.csv"
train_files = Path.cwd().glob(train_path)

train = pd.concat(
    (read_csv_pyarrow(file) for file in train_files),
    ignore_index=True,
)

train.head(2)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,acf7fbca426d323f580adddb4fa0f73598294df91cdaee...,Non-Logged,1,d6644c32-5828-4f88-ba44-a8d74c01d7bf,1657303857108,24,48232,47.5,1,1657303857108
1,0972299721e32759020a313bd97dcee19227fc3dd23164...,Non-Logged,1,68510c52-5530-4c89-9a8a-0ee36ab61546,1660338410977,0,10000,9.24,1,1660338410977


In [5]:
train.describe(include="all")

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
count,577942,577942,577942.0,577942,577942,577942,577942,577942,577942,577942
unique,577942,2,<NA>,340320,577934,182881,384167,325080,72321,577934
top,acf7fbca426d323f580adddb4fa0f73598294df91cdaee...,Non-Logged,<NA>,45a23540-7c1b-4d3f-89bc-74d897e2a489,1660335071796,0,10000,12.5,1,1660335071796
freq,1,458856,<NA>,1407,2,96760,31860,3043,240117,2
mean,NaN,NaN,14.056689,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,46.037793,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577942 entries, 0 to 577941
Data columns (total 10 columns):
 #   Column                   Non-Null Count   Dtype          
---  ------                   --------------   -----          
 0   userId                   577942 non-null  string[pyarrow]
 1   userType                 577942 non-null  string[pyarrow]
 2   historySize              577942 non-null  int64[pyarrow] 
 3   history                  577942 non-null  string[pyarrow]
 4   timestampHistory         577942 non-null  string[pyarrow]
 5   numberOfClicksHistory    577942 non-null  string[pyarrow]
 6   timeOnPageHistory        577942 non-null  string[pyarrow]
 7   scrollPercentageHistory  577942 non-null  string[pyarrow]
 8   pageVisitsCountHistory   577942 non-null  string[pyarrow]
 9   timestampHistory_new     577942 non-null  string[pyarrow]
dtypes: int64[pyarrow](1), string[pyarrow](9)
memory usage: 743.7 MB


Temos 10 colunas diferentes, sendo elas, segundo a própria documentação da FIAP:

- `userId`: id do usuário.
- `userType`: usuário logado ou anônimo.
- `historySize`: quantidade de notícias lidas pelo usuário.
- `history`: lista de notícias visitadas pelo usuário.
- `timestampHistory`: momento em que o usuário visitou a página.
- `timeOnPageHistory`: quantidade de ms em que o usuário ficou na página.
- `numberOfClicksHistory`: quantidade de clicks na matéria.
- `scrollPercentageHistory`: quanto o usuário visualizou da matéria.
- `pageVisitsCountHistory`: quantidade de vezes que o usuário visitou a matéria.

Por mais que não esteja no enunciado do projeto, também existe a coluna `timestampHistory_new` no dataset, mas parece ser um erro de coluna duplicada, já que seu valor é idêntico ao da coluna `timestampHistory`.

In [7]:
display(
    "Há algum valor diferente quando comparamos as `timestampHistory`?",
    (train["timestampHistory"] != train["timestampHistory_new"]).any(),
)

'Há algum valor diferente quando comparamos as `timestampHistory`?'

False

Sendo assim, iremos remover a coluna:

In [8]:
train = train.drop(columns=["timestampHistory_new"])
train.head(1)

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
0,acf7fbca426d323f580adddb4fa0f73598294df91cdaee...,Non-Logged,1,d6644c32-5828-4f88-ba44-a8d74c01d7bf,1657303857108,24,48232,47.5,1


**Temos usuários iguais/duplicados no dataset?**\
Dependendo da forma que o G1 lida com os dados, talvez existam usuários com `userId` iguais, mas cada um com um `userType` diferente.

In [9]:
display(train.duplicated().sum())
display(train.duplicated(subset=["userId"]).sum())

np.int64(0)

np.int64(0)

O CSV apenas separa por vírgulas as colunas que são listas. Isso faz com que elas sejam interpretadas pelo tipo "string" quando são lidar pela engine do Pandas com PyArrow. Sendo assim, é bom convertê-las para conseguirmos utilizar métodos de listas nelas nas análises.

In [10]:
list_columns = [
    "history",
    "timestampHistory",
    "numberOfClicksHistory",
    "timeOnPageHistory",
    "scrollPercentageHistory",
    "pageVisitsCountHistory",
]

train[list_columns] = train[list_columns].apply(lambda x: x.str.split(", "))

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 577942 entries, 0 to 577941
Data columns (total 9 columns):
 #   Column                   Non-Null Count   Dtype                      
---  ------                   --------------   -----                      
 0   userId                   577942 non-null  string[pyarrow]            
 1   userType                 577942 non-null  string[pyarrow]            
 2   historySize              577942 non-null  int64[pyarrow]             
 3   history                  577942 non-null  list<item: string>[pyarrow]
 4   timestampHistory         577942 non-null  list<item: string>[pyarrow]
 5   numberOfClicksHistory    577942 non-null  list<item: string>[pyarrow]
 6   timeOnPageHistory        577942 non-null  list<item: string>[pyarrow]
 7   scrollPercentageHistory  577942 non-null  list<item: string>[pyarrow]
 8   pageVisitsCountHistory   577942 non-null  list<item: string>[pyarrow]
dtypes: int64[pyarrow](1), list<item: string>[pyarrow](6), string

O usuário só pode estar logado ou deslogado (usuário anônimo/sem uma conta). Com essa informação, futuramente este campo pode ser tratado para virar um inteiro, sendo 1 (logados) e 0 (deslogados).

O dataset está longe de estar balanceado pelo tipo do usuário. Tem-se quase 80% da base com usuários `Non-Loged`, isso já nos mostra que muitas pessoas acessam as notícias sem login ou criação de uma conta.

In [12]:
train["userType"].value_counts(normalize=True) * 100

userType
Non-Logged    79.394818
Logged        20.605182
Name: proportion, dtype: double[pyarrow]

**Como é o comportamento dos valores?**\
Como os valores estão em formato de listas, utilizar o comando "describe" não é o bastante para gerar as métricas básicas de cada coluna.

Conforme podemos ver abaixo, o todas as colunas possuem valores bem mais altos que o normal, provavelmente representando algum tipo de erro ou um acesso fora do comum (outlier) no dataset.

- O resto das colunas acaba somando suas quantidades de acordo com `pageVisitsCountHistory` do usuário. Por exemplo: o valor máximo da coluna `numberOfClicksHistory` é 14562, e ao investigar este valor, a página em questão é um **dashboard interativo** e o usuário acessou a página **29 vezes**, explicando valores altos de clicks, de `timeOnPageHistory` (1198939).
- A coluna `scrollPercentageHistory` deveria, por lógica, ir de 0 até 100, já que é o percentual do quanto o usuário visualizou a página (0 = não visualizou nada; 100 = visualizou tudo), porém é possível ver valores enormes nessa coluna. Sendo assim, um tratamento aqui seria normalizar o valor, transformando valores acima de 100 no número 100.
  - Outra coisa que foi possível perceber é que não necessariamente métricas altas (como um longo tempo de acesso na página, muitos clicks, etc.) indicam que o usuário tem um percentual de scroll alto. Isso já indica que não podemos levar essa coluna tão a sério assim, pois parece não monitorar tão bem o usuário.

In [58]:
with pd.option_context("display.float_format", "{:.2f}".format):
    display(pd.to_numeric(train["numberOfClicksHistory"].explode()).describe())

count   8123951.00
mean         12.05
std          37.39
min           0.00
25%           0.00
50%           1.00
75%          14.00
max       14562.00
Name: numberOfClicksHistory, dtype: double[pyarrow]

In [57]:
with pd.option_context("display.float_format", "{:.2f}".format):
    display(pd.to_numeric(train["pageVisitsCountHistory"].explode()).describe())

count   8123951.00
mean          1.14
std           1.08
min           1.00
25%           1.00
50%           1.00
75%           1.00
max         654.00
Name: pageVisitsCountHistory, dtype: double[pyarrow]

In [55]:
with pd.option_context("display.float_format", "{:.2f}".format):
    display(pd.to_numeric(train["timeOnPageHistory"].explode()).describe())

count    8123951.00
mean       88768.69
std       113932.18
min         5000.00
25%        29870.00
50%        60000.00
75%       114582.00
max     46033049.00
Name: timeOnPageHistory, dtype: double[pyarrow]

In [56]:
with pd.option_context("display.float_format", "{:.2f}".format):
    display(pd.to_numeric(train["scrollPercentageHistory"].explode()).describe())

count   8123951.00
mean         54.01
std        1832.26
min           0.02
25%          27.08
50%          43.23
75%          59.93
max      526275.56
Name: scrollPercentageHistory, dtype: double[pyarrow]

**Todas as listas possuem o mesmo tamanho?**\
Como elas falam sobre o histórico do usuário, o correto é que cada lista em cada linha tenha o mesmo tamanho que aparece na coluna `historySize`.

- Sim! Fizemos uma função que compara o tamanho em `historySize` com o tamanho das listas que temos no dataframe. Ela retornou `True`, indicando que todas as listas possuem o tamanho correto.


In [12]:
def check_size(row: pd.Series, size: int, list_columns: list[str]) -> bool:
    """Check if the len of `historySize` is equal to the len of other columns.

    Args:
        row (pd.Series): the row containing the columns in `list_columns`
        size (int): the size to be checked
        list_columns (list[str]): list with column names of the dataframe

    Returns:
        bool: result of the comparison, returns True if equal.

    """
    return all(len(row[col_list]) == size for col_list in list_columns)


all(train.apply(lambda x: check_size(x, x["historySize"], list_columns), axis=1))

True

**Todos os usuários possuem ao menos um valor em seu histórico de acesso?**\
Isso é relevante para entender quando o G1 começa a monitorar o usuário: é assim que ele entra em qualquer página ou somente quando acessa sua primeira notícia?

Vendo que todos os usuários possuem algum valor em seu `history`, isso já fica claro que é a partir da primeira notícia que o monitoramento começa. Todos os usuários possuem algum histórico, mesmo que seja de apenas uma notícia.

In [13]:
all(train["history"].apply(len) > 0)

True

In [14]:
train["historySize"].describe()

count     577942.0
mean     14.056689
std      46.037793
min            1.0
25%            1.0
50%            2.0
75%            6.0
max         7004.0
Name: historySize, dtype: double[pyarrow]

In [15]:
train["historySize"].median()

2.0

O tamanho do histórico possui 14 de média e 2 de mediana, indicando que alguns usuários possuem um valor de histórico bem alto na base. Os percentis baixos junto com o tamanho máximo alto também indica isso.

Infelizmente, como não há uma coluna indicando a data de início que o usuário começou a ter seu histórico monitorado, não temos como saber se o tamanho do histórico dele é proporcional ao tempo que ele acessa o G1.

In [16]:
BIG_HIST_SIZE = 800
display(
    f"Há {(train['historySize'] >= BIG_HIST_SIZE).sum()} usuários com um histórico \
maior ou igual que {BIG_HIST_SIZE}. Temos {train['userId'].count()} usuários na base."
)

train.sort_values(by="historySize", ascending=False).head(5)

'Há 135 usuários com um histórico maior ou igual que 800. Temos 577942 usuários na base.'

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
433750,1b9bf55cf5c0d7ba34e8925dbbc98da17c5ab360d907c5...,Non-Logged,7004,['26b69894-5e12-4e11-b36a-94e6ec3172a0' '9277...,['1658713338344' '1658713440930' '165871481029...,['0' '20' '1' ... '0' '0' '0'],['136395' '290000' '160971' ... '10000' '10000...,['15.77' '12.72' '4.26' ... '6.19' '6.38' '7.95'],['5' '5' '9' ... '1' '1' '1']
347719,1f7a8e71d6d871e6a77f193c0cece8a6ebcd304dcfddcc...,Logged,5907,['f609464e-5777-4bc9-828a-47f982cc3fed' '007f...,['1656644949511' '1656645019617' '165664533024...,['0' '0' '0' ... '0' '0' '0'],['40000' '28550' '98640' ... '50000' '25196' '...,['43.16' '38.38' '16.89' ... '43.98' '35.02' '...,['1' '1' '5' ... '1' '1' '1']
251786,83a2ba48f07e49f33ef4aa3ceb90933f9adff1ddb6ed16...,Non-Logged,4914,['4745a8d7-9f4f-4beb-8b8d-5adb92000082' '1615...,['1656672728140' '1656672826027' '165667447595...,['0' '0' '0' ... '2' '2' '0'],['10000' '10000' '27069' ... '90000' '40000' '...,['6.88' '6.28' '25.96' ... '44.79' '11.4' '14....,['1' '1' '1' ... '1' '1' '1']
386620,d3b82a1305105bcaaa646d2da1e97b6935deddbcfbd12b...,Logged,3580,['eaea3d49-6fc3-475d-ba30-fa02a91daa98' '4e1d...,['1656645129077' '1656645137042' '165664523380...,['0' '1' '8' ... '17' '1' '0'],['10000' '23204' '100000' ... '110000' '33506'...,['13.3' '27.72' '27.03' ... '40.09' '18.55' '2...,['1' '1' '2' ... '1' '1' '1']
136184,9d176e2202250e7925b17eae0978eee726f8ff9372e45c...,Non-Logged,3318,['07ee94b0-9c39-448e-b96c-3d67f9c1670f' 'fad4...,['1656691004852' '1656691099012' '165669126445...,['24' '55' '8' ... '16' '63' '72'],['89910' '156859' '20274' ... '39313' '114801'...,['72.44' '79.86' '35.65' ... '58.53' '98.72' '...,['1' '1' '1' ... '1' '1' '1']


**Uma mesma notícia pode aparecer mais de uma vez no histórico do usuário?**\
Como o G1 lida com um usuário que acessa uma mesma notícia mais de uma vez?

O usuário abaixo tem um histórico com quatro notícias. Dentre elas, pela coluna `pageVisitsCountHistory` ele acessou a primeira página 12 vezes, a segunda 2 vezes, a terceira 1 vez e a quarta 4 vezes.

Analisando as outras colunas de histórico, chegamos a algumas conclusões:
- O `history` e nem as outras colunas aumentam de tamanho, mesmo ele tendo visto as páginas mais vezes
- O `timestampHistory` segue a ordem do primeiro acesso do usuário, ou seja, o 1° valor da lista é o menor que o segundo, o 2° é menor do que o terceiro, e assim vai até o último
- Tudo indica que o `numberOfClicksHistory` é um somatório dos clicks em cada um dos acessos na página. Um forte indicativo disso é que a 3° página diz que recebeu apenas seis clicks e foi acessada uma vez, mas as outras receberam vários outros clicks.
- Assim como o `numberOfClicksHistory`, o `timeOnPageHistory` parece funcionar da mesma forma, somando o tempo que o usuário passou em cada acesso.
- Diferente dos itens acima, o `scrollPercentageHistory` parece não agir da mesma forma, pois é extremamente improvável que o usuário tenha apenas "20.83%" de acesso na primeira página, sendo que a visitou 12 vezes, passou o maior tempo nesta página do que as outras e deu 127 clicks durante suas visitas.

Também algumas conclusões sobre os tipos:
- `timestampHistory` pode ser convertido para a data real - talvez isso poderia indicar uma certa sazonalidade em algumas notícias. Notícias sobre o Natal, por exemplo, provavelmente são mais acessadas durante os meses de novembro, dezembro e janeiro.
- `timeOnPageHistory` representa, provavelmente, o tempo na página em milissegundos (ms), já que se medirmos esses tempos em segundos ou microsegundos, as medidas ficariam improváveis para um caso real

In [17]:
with pd.option_context("display.max_colwidth", 1):
    display(
        train.query(
            "userId=='6aa5109374f6534b0400ccbbdb9c7b7d64c40a663fe51c95102754e28948f92e'"
        )
    )

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
377947,6aa5109374f6534b0400ccbbdb9c7b7d64c40a663fe51c...,Non-Logged,4,['7fe849c0-4a55-429d-b480-11ee216909dd' '2de3...,['1658259718982' '1658424852871' '165885911024...,['127' '155' '6' '78'],['350199' '320000' '39671' '151452'],['20.83' '80.94' '45.18' '22.32'],['12' '2' '1' '4']


In [18]:
# Comparison of timestampHistory
(1658259718982 < 1658424852871) and (1658424852871 < 1658859110243) and (  # noqa: PLR0133
    1658859110243 < 1660064956101  # noqa: PLR0133
)

True

As unidades devem estar em milissegundos (ms), pois é a unidade que mais faz sentido:

In [19]:
pd.to_datetime([1658259718982, 1658424852871, 1658859110243, 1660064956101], unit="ms")

DatetimeIndex(['2022-07-19 19:41:58.982000', '2022-07-21 17:34:12.871000', '2022-07-26 18:11:50.243000', '2022-08-09 17:09:16.101000'], dtype='datetime64[ns]', freq=None)

## Análise dos dados de validação

In [20]:
validation_path = Path("../data/raw/validacao.csv")
validation = read_csv_pyarrow(validation_path)
validation.head(2)

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,[1660533136590 1660672113513]
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],[1660556860253]


A primeira coisa que notamos no dataset é a **falta de colunas em relação aos dados de treino**. Tem-se apenas quatro colunas, que significam a mesma coisa do dataset de treino:

- `userId`: id do usuário.
- `userType`: usuário logado ou anônimo.
- `history`: lista de notícias visitadas pelo usuário.
- `timestampHistory`: momento em que o usuário visitou a página.

In [21]:
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112184 entries, 0 to 112183
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype          
---  ------            --------------   -----          
 0   userId            112184 non-null  string[pyarrow]
 1   userType          112184 non-null  string[pyarrow]
 2   history           112184 non-null  string[pyarrow]
 3   timestampHistory  112184 non-null  string[pyarrow]
dtypes: string[pyarrow](4)
memory usage: 19.3 MB


Também não temos dados duplicados

In [22]:
display(validation.duplicated().sum())
display(validation.duplicated(subset=["userId"]).sum())

np.int64(0)

np.int64(0)

Diferente do dataset de treino, no de validação temos um dataset mais balanceado pelo tipo de usuário.

Novamente, isso não importa tanto assim, já que o objetivo do projeto visa atender todos os tipos de usuários igualmente: recomendando as notícias mais apropriadas.

In [23]:
validation["userType"].value_counts(normalize=True) * 100

userType
Non-Logged    58.176745
Logged        41.823255
Name: proportion, dtype: double[pyarrow]

Novamente, por conta da forma que os dados são lidos, precisamos transformar as colunas para uma lista de strings.

Obs.: foi necessário tratar melhor os dados, pois eles possuem uma formatação um pouco diferente do que tinham no arquivo de treino

In [24]:
list_columns = ["history", "timestampHistory"]

validation["history"] = (
    validation["history"]
    .str.replace(r"[\[\]']", "", regex=True)
    .str.replace(r"\n", "", regex=True)
    .str.split()
)
validation["timestampHistory"] = (
    validation["timestampHistory"].str.replace(r"[\[\]]", "", regex=True).str.split()
)
validation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112184 entries, 0 to 112183
Data columns (total 4 columns):
 #   Column            Non-Null Count   Dtype                      
---  ------            --------------   -----                      
 0   userId            112184 non-null  string[pyarrow]            
 1   userType          112184 non-null  string[pyarrow]            
 2   history           112184 non-null  list<item: string>[pyarrow]
 3   timestampHistory  112184 non-null  list<item: string>[pyarrow]
dtypes: list<item: string>[pyarrow](2), string[pyarrow](2)
memory usage: 19.8 MB


E, igual com o dataset de treino, as colunas de histórico possuem o mesmo tamanho - o que é o correto.

> Como não temos a coluna `historySize`, aqui apenas comparamos os tamanhos das duas colunas

In [25]:
display(validation.head(2))
display("Tamanho de history:", validation["history"].head(2).apply(len))

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c5...,['1660533136590' '1660672113513']
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],['1660556860253']


'Tamanho de history:'

0    2
1    1
Name: history, dtype: int64

In [26]:
all(validation["history"].apply(len) == validation["timestampHistory"].apply(len))

True

Quando verificamos os IDs da tabela de validação dentro da tabela de treino, vemos que eles existem nas duas tabelas, porém, na de treino, a informação é mais completa do que na de validação.

In [27]:
all(validation["userId"].isin(train["userId"]))

True

In [28]:
display(f"Primeiro userId na validation: f{validation['userId'][0]}")

with pd.option_context("display.max_colwidth", 1):
    display("Validação:", validation.head(1))
    display("Treino:", train.query(f"userId == '{validation['userId'][0]}'"))

'Primeiro userId na validation: fe25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9'

'Validação:'

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c59ff6-fb82-4258-918f-2910cb2d4c52'],['1660533136590' '1660672113513']


'Treino:'

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
482774,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4...,Logged,6,['c7b6f56c-3304-4fe5-994a-50c8937d6431' 'c6af...,['1657003078783' '1659484116867' '165970076624...,['5' '1' '10' '16' '4' '2'],['20848' '10000' '37197' '90813' '23755' '1000...,['36.51' '19.78' '46.59' '36.45' '32.46' '13.89'],['1' '1' '1' '1' '1' '1']


In [29]:
pd.to_datetime(
    [
        1660533136590,  # validação
        1660672113513,  # validação
        1657003078783,  # treino
        1659484116867,  # treino
        1659700766244,  # treino
        1659929500885,  # treino
        1660062239512,  # treino
        1660182601797,  # treino
    ],
    unit="ms",
)

DatetimeIndex(['2022-08-15 03:12:16.590000', '2022-08-16 17:48:33.513000', '2022-07-05 06:37:58.783000', '2022-08-02 23:48:36.867000', '2022-08-05 11:59:26.244000', '2022-08-08 03:31:40.885000', '2022-08-09 16:23:59.512000', '2022-08-11 01:50:01.797000'], dtype='datetime64[ns]', freq=None)

In [30]:
with pd.option_context("display.max_colwidth", 1):
    display("Validação:", validation.head(2))
    display("Treino:", train.query(f"userId == '{validation['userId'][1]}'"))

'Validação:'

,userId,userType,history,timestampHistory
0,e25fbee3a42d45a2914f9b061df3386b2ded2d8cc1f3d4b901419051126488b9,Logged,['be89a7da-d9fa-49d4-9fdc-388c27a15bc8' '01c59ff6-fb82-4258-918f-2910cb2d4c52'],['1660533136590' '1660672113513']
1,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdeee870d49b897e1e99cd,Logged,['77901133-aee7-4f7b-afc0-652231d76fe9'],['1660556860253']


'Treino:'

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
554897,d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdee...,Logged,1,['9a37393c-15fc-413a-b285-860da171d0a2'],['1656668214243'],['15'],['30883'],['35.35'],['1']


In [31]:
# validação e treino
pd.to_datetime([1660556860253, 1656668214243], unit="ms")

DatetimeIndex(['2022-08-15 09:47:40.253000', '2022-07-01 09:36:54.243000'], dtype='datetime64[ns]', freq=None)

Pela análise acima, percebe-se algo estranho:
- O `history` **não contém** as mesmas notícias nos dois datasets, mas é o mesmo usuário.
- O `timestampHistory` segue da mesma forma, não registrou as notícias corretamente.

Retirando dúvidas com a FIAP e analisando melhor os datasets, percebemos que **é o mesmo usuário**, porém com seu histórico de notícias sendo capturado em um período diferente.

Exemplo com o usuário de ID "d0afad7ea843d86597d822f0df1d39d31a3fea7c39fdeee870d49b897e1e99cd":
- No dataset de treino, ele possui uma notícia em seu histórico. Ela foi acessada na data 2022-07-01
- Já no dataset de validação, ele possui uma notícia também, mas ela foi acessada na data 2022-08-15

O mesmo ocorre para outros usuários. Isso nos mostra que o dataset de treino captura as notícias **mais antigas**, enquanto o de validação irá capturar as **mais recentes** consumidas pelo usuário.

Sendo assim, a forma correta de validar nosso modelo será:
1. Buscamos no dataset de treino todas as linhas que contém os IDs que estão no dataset de validação
2. Com essas linhas (do dataset de treino), iremos usar nosso modelo para recomendar/prever as N próximas notícias que esses usuários devem consumir
   1. O número de previsões (N) pode ser fixo, mas é melhor que seja dinâmico, onde o valor de N irá ser definido pelo tamanho do histórico do usário no dataset de validação. Ou seja, se o usuário tem um `history` de três notícias no dataset de validação, o valor de N deve ser 3.
3. Iremos pegar as notícias previstas e comparar com as que estão no dataset de validação
4. Com essa comparação, podemos criar algum tipo de score (ainda será definido) para validar se o modelo está prevendo bem ou não.

## Análise dos itens complementares

In [65]:
itens_path = "../data/raw/itens/itens/*.csv"
itens_files = Path.cwd().glob(itens_path)

itens = pd.concat(
    (read_csv_pyarrow(file) for file in itens_files),
    ignore_index=True,
)

itens.head(2)

,page,url,issued,modified,title,body,caption
0,13db0ab1-eea2-4603-84c4-f40a876c7400,http://g1.globo.com/am/amazonas/noticia/2022/0...,2022-06-18 20:37:45+00:00,2023-04-15 00:02:08+00:00,Caso Bruno e Dom: 3º suspeito tem prisão tempo...,"Após audiência de custódia, a Justiça do Amazo...",Jeferson da Silva Lima foi escoltado por agent...
1,92907b73-5cd3-4184-8d8c-e206aed2bf1c,http://g1.globo.com/pa/santarem-regiao/noticia...,2019-06-20 17:19:52+00:00,2023-06-16 20:19:15+00:00,Linguajar dos santarenos é diferenciado e chei...,Vista aérea de Santarém Ádrio Denner/ AD Produ...,As expressões santarenas não significam apenas...


- `page`: ID da matéria
- `url`: URL da matéria
- `issued`: data que a matéria foi criada
- `modified`: última data que a matéria foi modificada
- `title`: título da matéria
- `body`: corpo da matéria
- `caption`: subtítulo da matéria

Para os itens, a URL nos dá uma certa informação sobre o tipo da notícia. Ex.:
- http://g1.globo.com/am/amazonas/noticia/2022/06/18/(...)
  - A URL fornece a data que a notícia foi postada (18/06/2022)
  - A URL fornece as "tags" da notícia:
    - É do Amazonas, por isso contém 'am'
    - A notícia é específica de Amazonas, por isso contém "amazonas"
    - É uma notícia, então fica com "noticia" na tag
- A URL "http://g1.globo.com/pa/santarem-regiao/noticia/2019/06/20/(...)" funciona de forma igual ao exemplo acima
- http://g1.globo.com/mundo/noticia/2022/07/08/
  - A notícia é internacional, então recebe a tag "mundo"
  - Diferente das outras, não teve uma tag específica para a região que ocorreu. Isso já indica uma certa probabilidade de que o G1 recomende notícias para as pessoas baseadas na sua localização, infelizmente é um dado que não temos
- http://g1.globo.com/politica/noticia/2021/09/09/(...)
  - Aqui a tag mudou, por mais que seja algo brasileiro, apenas ficou como "politica"
- http://g1.globo.com/politica/eleicoes/2022/noticia/2022/04/18/(...)
  - Aqui a tag ficou mais específica, ainda está dentro de "politica", mas faz parte das eleições
  - O "2022" aparece duas vezes, mas tem significados diferentes, um deles indica que a notícia é sobre as eleições de 2022, não necessariamente fala sobre a data de publicação da notícia

Dito isso, temos o seguinte padrão:
1. Aparece a URL base do G1 (http://g1.globo.com/)
2. Aparece uma longa URL (tamanho variado) que fornece tags sobre a notícia, como sua região (estado, cidade ou se é internacional) ou seu tema (política, eleições, economia, etc.)
3. Aparece a data da postagem da notícia - que é o mesmo valor que aparece na coluna `issued`
4. E, por fim, tem-se a página e sua extensão (ghtml, por exemplo). Essa parte pode servir como a coluna `title` já tratada, pois segue o mesmo padrão, mas sem acentos. Ex.:
   - Na url: "eleicoes-titulo-de-eleitor-cancelado-o-que-fazer-saiba-como-regularizar-a-tempo-de-votar.ghtml"
   - No title: "Eleições: título de eleitor cancelado, o que fazer? Saiba como regularizar a tempo de votar"
   - Com isso, pode-se utilizar a URL para pular a etapa de tratamento do título, pois a URL já aplica:
     - Remoção de caracteres especiais (pontos, vírgulas etc.)
     - Substituição de acentos pelo caractere de origem: "ç" vira "c", "í" vira "i"
     - Tudo fica em letras minúsculas

Agora, sobre as outras colunas, a mais preocupante parece a `body` - o corpo da notícia.

Ela contém um enorme texto e precisará de uma série de tratamentos para ser processada corretamente em algoritmos de machine learning.

In [33]:
itens["title"][0]

'Caso Bruno e Dom: 3º suspeito tem prisão temporária decretada pela Justiça do AM'

In [34]:
itens["caption"][0]

'Jeferson da Silva Lima foi escoltado por agentes da Polícia Federal ao Fórum de Justiça do município para a audiência de custódia'

In [35]:
itens["body"][0]

'Após audiência de custódia, a Justiça do Amazonas decretou, na tarde deste sábado (18), a prisão temporária,  por 30 dias, de Jeferson da Silva Lima, conhecido como "Pelado da Dinha". Ele teve participação direta na morte do indigenista Bruno Pereira e do jornalista inglês Dom Phillips, aponta as investigações. \n"Pelado da Dinha" foi considerado foragido na noite de sexta-feira (17) após ter o mandado de prisão expedido e não ser localizado pelas autoridades. Ele se entregou na delegacia de Atalaia do Norte, a 1.136 quilômetros de Manaus, nas primeiras horas da manhã deste sábado, onde foi ouvido pelo delegado Alex Perez Timóteo. \n"Pelado da Dinha" foi considerado foragido na noite de sexta-feira (17) após ter o mandado de prisão expedido \nRôney Elias/Rede Amazônica \nDurante a tarde, Jeferson foi escoltado por agentes da Polícia Federal ao Fórum de Justiça do município para a audiência de custódia e teve a prisão temporária decretada. \nPerícia confirma identificação dos restos mo

In [44]:
itens.duplicated().sum()

np.int64(0)

In [43]:
itens.duplicated(subset=["page"]).sum()

np.int64(0)

In [ ]:
itens["issued"].describe()

count                       255603
mean     2021-02-01 03:10:26+00:00
min      2014-07-31 08:00:01+00:00
25%      2020-02-17 08:08:03+00:00
50%      2021-08-21 19:34:28+00:00
75%      2022-05-03 02:00:19+00:00
max      2022-08-15 02:57:16+00:00
Name: issued, dtype: object

In [ ]:
itens["issued"].value_counts().sort_values()

issued
2022-06-18 20:37:45+00:00      1
2019-06-20 17:19:52+00:00      1
2022-07-08 08:55:52+00:00      1
2021-09-09 19:06:46+00:00      1
2021-09-15 19:16:13+00:00      1
                            ... 
2021-01-07 17:29:00+00:00      5
2015-06-30 07:00:00+00:00      8
2015-12-29 07:00:00+00:00     22
2017-01-01 07:00:00+00:00     40
2018-10-07 20:00:00+00:00    248
Name: count, Length: 253864, dtype: int64[pyarrow]

**Todas as páginas aparecem no histórico de pelo menos um usuário?**

Conforme o código abaixo, sim, todas as páginas aparecem ao menos uma vez no histórico de algum usuário. Isso significa que não poderemos excluir nenhuma das páginas de nossa pesquisa, pois, por mais que o usuário tenha acessado uma página inválida, não podemos ignorar as preferências dele.

In [52]:
all_history_values = set()

for history_list in train["history"]:
    all_history_values.update(history_list)

for history_list in validation["history"]:
    all_history_values.update(history_list)

In [54]:
itens[~itens["page"].isin(all_history_values)]

,page,url,issued,modified,title,body,caption


**Todas as URLs são mesmo de notícias?**\
Em teoria, o dataset deveria ser composto 100% de dados referentes a notícias do G1, sem falar sobre outros tipos de sites (como uma página de FAQ ou a homepage, por exemplo)

Várias URLs possuem nela o texto/tag "noticia", indicando que aquela página do G1 é uma notícia.

In [36]:
with pd.option_context("display.max_colwidth", 1):
    display(itens.head(5)["url"])

0    http://g1.globo.com/am/amazonas/noticia/2022/06/18/caso-bruno-e-dom-3o-suspeito-tem-prisao-temporaria-decretada-pela-justica-do-am.ghtml                        
1    http://g1.globo.com/pa/santarem-regiao/noticia/2019/06/20/linguajar-dos-santarenos-e-diferenciado-e-cheio-de-identidade-egua-tu-nao-vai-ser-leso-de-perder.ghtml
2    http://g1.globo.com/mundo/noticia/2022/07/08/ex-premie-shinzo-abe-morre-apos-ser-baleado-no-japao-diz-nhk.ghtml                                                 
3    http://g1.globo.com/politica/noticia/2021/09/09/relator-no-stf-fachin-vota-contra-marco-temporal-para-demarcacao-de-terras-indigenas.ghtml                      
4    http://g1.globo.com/politica/noticia/2021/09/15/apos-2-votos-pedido-de-vista-suspende-julgamento-no-stf-sobre-demarcacao-de-terras-indigenas.ghtml              
Name: url, dtype: string[pyarrow]

Abaixo, pode-se ver que nem todas possuem a tag "noticia" em sua URL.

In [37]:
itens["url"].str.contains("/noticia/").value_counts()

url
True     250533
False      5070
Name: count, dtype: int64[pyarrow]

Temos páginas que **não são notícias**, como a página institucional que fala sobre a equipe do G1.

Além disso, muitas notícias foram marcadas com a tag "blog" ao invés de "notícia", indicando que, por mais que seja uma notícia, ela faz parte de um blog específico. Aparentemente, uma notícia/postagem de um blog é feita para agrupar notícias que foram postadas por uma pessoa.

> Ex.: Temos um blog chamado "Viver Noronha", ele pode ser identificado vendo se a URL contém "/blog/viver-noronha/".

No enriquecimento de features, isso pode virar uma tag chamada "blog-viver-noronha", pois algumas pessoas podem estar mais interessadas em notícias desse blog.

In [ ]:
with pd.option_context("display.max_colwidth", 1):
    display(itens[~itens["url"].str.contains("/noticia/")]["url"])

8         http://g1.globo.com/institucional/equipe-do-g1.ghtml                                                                                                                                      
319       http://g1.globo.com/pe/pernambuco/blog/viver-noronha/post/2022/05/23/stf-marca-audiencia-de-conciliacao-entre-uniao-e-pernambuco-na-disputa-pela-titularidade-de-fernando-de-noronha.ghtml
320       http://g1.globo.com/pe/pernambuco/blog/viver-noronha/post/2022/06/11/curso-gratuito-de-cinema-e-oferecido-para-moradores-de-fernando-de-noronha.ghtml                                     
732       http://g1.globo.com/politica/blog/andreia-sadi/post/andreia-sadi.ghtml                                                                                                                    
733       http://g1.globo.com/politica/blog/julia-duailibi/post/2022/05/23/doria-comunicou-desistencia-da-pre-candidatura-da-presidencia-a-aliados-no-domingo.ghtml                                 
               

: 

Mesmo sem considerar "noticia", "blog" ou "post" nas URLs, ainda temos algumas URLs que são notícias, tornando difícil de identificar quais URLs são ou não notícias em nosso dataset.

Identificamos abaixo:
- Há "institucional" em URLs que não são notícias, apenas uma notícia institucional do site;
- Há "especiais.g1.globo.com" na URL de base para sites que não são uma notícia, mas sim algum tipo de matéria especial, como em "http://especiais.g1.globo.com/educacao/guia-de-carreiras/2017/teste-vocacional/", que é apenas um teste vocacional, e "http://especiais.g1.globo.com/", que é uma URL quebrada/não funcional;
- As tags "quiz" e "interatividade" são presentes em alguns sites que não são notícias, mas um conteúdo interativo

In [ ]:
with pd.option_context("display.max_colwidth", 1):
    display(
        itens[~itens["url"].str.contains("/noticia/|/blog/|/post/", regex=True)]["url"]
    )

Removendo todas os textos identificados acima que indicam que algo não é uma notícia, sobraram poucos resultados no filtro, indicando que eles **são notícias**, mas que tem uma estrutura da tag diferente do convencional.

Sendo assim, para remover da base as coisas que **não** são notícias, mantendo todo o resto, precisaremos remover URLs com:
- /especiais.g1.globo.com/
- /interatividade/
- /institucional/
- /quiz/

Sendo assim, ainda teremos matérias onde sua URL contém a estrutura mais presente na base (contendo "notícia"), mas manterá publicações em blogs, postagens sem um blog e as notícias que apareceram abaixo, que não seguem um formato específico de URL.

In [ ]:
with pd.option_context("display.max_colwidth", 1):
    display(
        itens[
            ~itens["url"].str.contains(
                "/especiais.g1.globo.com/|/noticia/|/blog/|/post/|/interatividade/|/institucional/|/quiz/",
                regex=True,
            )
        ]["url"]
    )

967       http://g1.globo.com/mg/minas-gerais/apos-dois-...
7742      http://g1.globo.com/mg/minas-gerais/belotur-pr...
24583     http://g1.globo.com/to/tocantins/segundo-turno...
28554     http://g1.globo.com/to/tocantins/ifto-abre-ext...
47641     http://g1.globo.com/ro/rondonia/rondonia-rural...
142212    http://g1.globo.com/sp/sao-paulo/carnaval/2022...
147912    http://g1.globo.com/ba/bahia/blogdoemprego/ass...
152209    http://g1.globo.com/rs/rio-grande-do-sul/banco...
193735    http://g1.globo.com/globonews/eleicoes-e-suste...
199999    http://g1.globo.com/pa/para/canaa-dos-carajas-...
226681    http://g1.globo.com/mundo/por-que-os-vikings-n...
234740    http://g1.globo.com/to/tocantins/policial-civi...
Name: url, dtype: string[pyarrow]

Porém, antes de pensarmos em manter apenas notícias válidas na base, precisamos descobrir se os usuários nos nossos datasets de treino e validação acessaram alguma dessas páginas.

Isso é muito importante, pois como estamos falando sobre fazer um **recomendador de notícias**, o ideal seria manter apenas as notícias válidas na base, porém, caso tenhamos também essas não-notícias na base, isso significa que nosso modelo precisará ser treinado com elas, mesmo **não sendo a intenção do projeto**.

In [ ]:
not_news_tags = "/especiais.g1.globo.com/|/interatividade/|/institucional/|/quiz/"

not_news = itens[itens["url"].str.contains(not_news_tags, regex=True)]
not_news.info()

<class 'pandas.core.frame.DataFrame'>
Index: 596 entries, 8 to 253431
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype                        
---  ------    --------------  -----                        
 0   page      596 non-null    string[pyarrow]              
 1   url       596 non-null    string[pyarrow]              
 2   issued    596 non-null    timestamp[s, tz=UTC][pyarrow]
 3   modified  596 non-null    timestamp[s, tz=UTC][pyarrow]
 4   title     596 non-null    string[pyarrow]              
 5   body      596 non-null    string[pyarrow]              
 6   caption   596 non-null    string[pyarrow]              
dtypes: string[pyarrow](5), timestamp[s, tz=UTC][pyarrow](2)
memory usage: 599.9 KB


In [ ]:
not_news_set = set(not_news["page"])
not_news_set

{'01baa090-3aad-49c5-932b-b65aa8a11d4b',
 '023d795e-f870-436c-abe8-3cd6ec81a976',
 '031ff7a1-0101-40f8-8f1e-e038e3a1cca2',
 '03cbecd0-1590-47c7-8993-2f9aa47cc72e',
 '03dd47cc-6b07-4aa7-8924-87a0c0cd0815',
 '04831657-dfdb-46ac-81e0-0b93a412a95c',
 '06347173-bc55-4e2f-9ea7-7673ac178809',
 '084da78e-fcf9-4d50-869a-ccc7be947d62',
 '09b30338-48b3-4a60-b8eb-38c69ea0f4d8',
 '0a19f025-d33a-4c03-a7a0-530a9effa033',
 '0b22f887-6c89-4d0b-9ee4-07253ca890be',
 '15163840-dba9-484e-b731-8b54fe735320',
 '158064c4-e715-4ddd-881f-a77f755aaa88',
 '160d065a-f1a2-4768-b514-959f184b5ad6',
 '16912729-9a79-4cf4-a739-52ff88b0d8c7',
 '181b0989-5d84-49a7-b109-f0ba225d9806',
 '1a2c95ef-6156-4a42-9a35-2b25d70d7e1a',
 '1c3b37b0-020f-4bde-bfb8-0b0aed13d93e',
 '1c5f2865-8ce9-4391-815d-1e3030dbf567',
 '1e150d5d-91ed-4a42-b31b-de06e4e48022',
 '21204c7f-cebf-46eb-9049-5ded7f4f6116',
 '22735c26-8d91-4093-81f1-d3cb2a2cb591',
 '22860df5-1722-4a02-9206-e11f30354d2d',
 '23a206eb-db00-4448-b729-1132b1996c32',
 '258332fe-69ec-

In [ ]:
train[train["history"].apply(lambda x: any(id_ in not_news_set for id_ in x))]

,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory
152,ece48284c7581bff3faeb85741fb95ae58fcd183af9cd7...,Non-Logged,2,['esid:conteudo_editorial_g1#materia#https://e...,['1657126383344' '1657126383344'],['2' '2'],['8014' '8014'],['16.8' '16.8'],['1' '1']
269,cbaf80887e193891d23924b379688ca256682a4897c830...,Non-Logged,3,['be696673-5f83-4400-ab93-f56f451e9e19' 'esid...,['1660151308670' '1660151308670' '1660152597360'],['10' '10' '0'],['60000' '60000' '40000'],['67.99' '67.99' '59.94'],['1' '1' '1']
280,eefead1452bc97f14f748a2fcfa4af485a19453408f33e...,Non-Logged,3,['esid:conteudo_editorial_g1#materia#https://e...,['1658359852692' '1658359896915' '1658359926825'],['15' '13' '11'],['30398' '26830' '31023'],['26.42' '32.56' '22.56'],['1' '1' '1']
305,7d077f992c2b9accc434de86b2d1ab521565c1c20abaae...,Non-Logged,3,['533b3efd-b7d4-418e-92a9-0a1d88440963' 'esid...,['1658018768392' '1658032318887' '1658188311214'],['3' '6' '15'],['10000' '10657' '49387'],['8.21' '19.5' '28.32'],['1' '1' '1']
423,caa44b2543ed6b99fa332cc3c3146beb2c10b6937b21e1...,Non-Logged,1,['esid:conteudo_editorial_g1#materia#https://e...,['1657312329245'],['8'],['56306'],['65.99'],['2']
...,...,...,...,...,...,...,...,...,...
577475,51ba118535e7635f8f4edb0cee083bcdedd6fb92ce3bc8...,Non-Logged,84,['1f2b9c2f-a2d2-4192-b009-09065da8ec23' '7430...,['1656666412459' '1656666605741' '165675748632...,['27' '27' '19' '14' '15' '19' '17' '0' '44' '...,['110846' '72033' '101986' '89456' '120000' '1...,['58.63' '57.44' '35.02' '36.41' '46.01' '48.2...,['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '...
577493,780b3ea4acfd65a80d87f1f61e31309ccaa2c333f0c39a...,Logged,319,['c5ea9fce-3afb-4041-87b7-bf755e96d402' 'cf7c...,['1656674553340' '1656674913760' '165668577560...,['31' '2' '13' '20' '14' '17' '9' '12' '25' '2...,['274271' '30000' '52760' '114805' '91588' '70...,['54.01' '26.29' '30.57' '49.53' '32.08' '45.2...,['1' '1' '1' '1' '1' '1' '1' '1' '2' '1' '1' '...
577495,998b655d2845a186c6753ee3f34a81af9eec695b199bfb...,Logged,33,['6f1a3d83-b2aa-4eae-98de-3b07ab0fdec3' '1c4d...,['1659290766899' '1659291377410' '165929187002...,['8' '0' '1' '0' '0' '0' '1' '0' '0' '1' '0' '...,['116720' '27134' '57234' '122383' '23112' '84...,['42.06' '34.06' '68.06' '47.06' '74.59' '72.1...,['3' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '...
577503,f55d54fcb5cd9e65a6a884a9fcf3a3a8fbad300b9210b8...,Logged,145,['4549c4eb-c949-47e2-a071-7de377677d6b' '3c78...,['1656670787153' '1656670986496' '165667124737...,['32' '27' '41' '14' '52' '3' '1' '0' '2' '6' ...,['117885' '140000' '110000' '50000' '330000' '...,['51.05' '53.79' '50.21' '43.18' '64.73' '7.86...,['1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '1' '...


In [ ]:
validation[validation["history"].apply(lambda x: any(id_ in not_news_set for id_ in x))]

,userId,userType,history,timestampHistory
17,8d53c0ac0094404c0bf1c9e5d8c25837b6ae111e8b5feb...,Logged,['esid:conteudo_editorial_g1#materia#https://e...,['1660660075462']
19,5c8352bd70309a0fa30b56f637927f9cc83414d1c153ca...,Logged,['e2418f4d-6928-481d-ae7b-0a5b42c63bf6' '1a73...,['1660659447173' '1660659547056' '1660659667887']
79,86e84a6997cb9b6b72d226cf892d57e83a96216443f7aa...,Logged,['esid:conteudo_editorial_g1#materia#https://e...,['1660564665041' '1660564665041' '166058446904...
93,2e2df6314cc333c8e844ab4c2dff9af7e2691936f3a460...,Logged,['0203a71d-4c14-4c16-8dba-e0f540f9c0f8' '5d3d...,['1660576129843' '1660593510133' '166067505083...
115,37920e6f27c91e885f8e6ddcde0b351f095ae5149b3100...,Logged,['dbb1f75b-afd3-4041-939a-5b4a0fd17376' 'esid...,['1660588141544' '1660700231730']
...,...,...,...,...
112066,4942607aa564730bc7626e9a78f3a536b658d9d1d17651...,Non-Logged,['esid:conteudo_editorial_g1#materia#https://e...,['1660609036705']
112067,aff0018336e41f00e004281ff3904ba6abb48c11c47332...,Non-Logged,['esid:conteudo_editorial_g1#materia#https://e...,['1660573449771']
112068,b8d776d2026aa117c0d5eaba82c6a133b6cd47caa3e7ab...,Non-Logged,['esid:conteudo_editorial_g1#materia#https://e...,['1660561940835']
112069,7e30ab7f8023dfd1b54785954fb6a54ec40281c0f3d937...,Non-Logged,['esid:conteudo_editorial_g1#materia#https://e...,['1660566331155']


Infelizmente, através das validações acima, foi possível descobrir que nossos datasets possuem dados indicando que usuários acessaram páginas que não são notícias.

Por mais que sejam poucos usuários, como isso aparece em nosso dataset de validação, significa que nosso modelo deve ser capaz de sugerir matérias além de notícias aos usuários.

A pergunta inversa também foi analisada: **mas e as páginas que nenhum usuário possui no histórico, faria sentido recomendá-las?**

Pensando no escopo fechado do projeto, onde a métrica final será definida pela quantidade de acertos (e sendo que temos o dataset de validação presente para isso), faria sentido remover essas páginas, porém, ao pensar em um modelo **pronto para produção**, temos que considerar todas as matérias possíveis - exemplo: caso uma notícia de esportes tenha sido acessado nenhuma vez, ela ainda deve aparecer para usuários que possuem uma forte afinidade com esportes, sendo assim, não faz sentido removê-la do treinamento do modelo.

notas

- o modelo deve sugerir também notícias que façam mais sentido com as últimas vistas
  - Quando um user vê uma notícia de política, é bem provável que as próximas também serão políticas? faça a análise
- 